In [2]:
# -*- coding: utf-8 -*-
# imports
import pandas as pd
import requests
import re
import os
from yaml import safe_load, dump
from bs4 import BeautifulSoup as bs
import random
from modules import get_mapping_dicts, find_csv_urls, csvs_to_pandas, fill_gaps, standardise_cell_values, write_csv


"""
sdg-csv-data-filler is the first module in a data pipeline to take
data from the SDG data repo and make it exportable as CSVW.
"""

# setting paths to directories and files
remote_data_url = "https://github.com/ONSdigital/sdg-data/tree/develop/data"
cwd = os.getcwd()
data_path = os.path.join(cwd, 'data')
out_path = os.path.join(cwd, 'out')
gap_filler_yam_path = (os.path.join
                       ("substitutions",
                        "gap_filler.yaml"))
header_mapping_yam_path = (os.path.join
                           ("substitutions",
                            "header_mapping.yaml"))

In [9]:
# def entry_point(data_url):
#     urls_gen = find_csv_urls(data_url)
#     with open('overrides_dict.yaml') as file:
#         generic_from_yam = safe_load(file)['generic_overrides']

#     for _url in urls_gen:
#         data_name = extract_name(_url)
#         df = csvs_to_pandas(_url)
#         if not df:
#             continue
#         df = fill_gaps(df, gap_filler_dict)
#         df = standardise_headers(df)
#         write_csv(df, out_path)

# if __name__ == "__main__":
#     entry_point(data_url=remote_data_url)

In [31]:
to_write = ""
remote_data_url = "https://github.com/ONSdigital/sdg-data/tree/develop/data"
csv_gen = find_csv_urls(remote_data_url)


for url in csv_gen:
    if "no data for this indicator yet" in str(bs(requests.get(url).text)):
        continue
    yam_txt=f"{url} : \n"
    df = pd.read_csv(url)
#     import ipdb; ipdb.set_trace()
    header_overide_txt = ""
    column_specific_value_override=""
    for column in df.columns:
        header_overide_txt+=f"""#Header override for \'{column}\'
    {column} :
        to : correct_header_for_{column.replace(" ","_")}\n"""
        column_specific_value_override+=f"""#Value overrides for column \'{column}\'
        FILL_NA: filler_value
        OldValue1: NewValue1
        OldValue2: NewValue2
        OldValue3: NewValue3\n\n"""
        yam_txt +=f"""
        {header_overide_txt}
        {column_specific_value_override}"""
    to_write += yam_txt

with open('overrides.yaml', 'w+') as y:
    y.write(to_write)


In [47]:
# Yaml dump version
csv_gen = find_csv_urls(remote_data_url)
yam_dict={'generic_overrides':
          {'FILL_NA' : f'{column.replace(" ","_")}_gap_filler_value',
          'OldValue1' : 'NewValue1',
          'OldValue2' : 'NewValue2',
          'OldValue3' : 'NewValue3'}}
for url in csv_gen:
    if "no data for this indicator yet" in str(bs(requests.get(url).text)):
        continue
    df = pd.read_csv(url)
#     import ipdb; ipdb.set_trace()
    column_specific_value_dict= {}
    for column in df.columns:
        if column == 'Value':
            continue
        column_specific_value_dict[column]= {'to' : f'correct_header_for_{column.replace(" ","_")}',
            'FILL_NA' : f'{column.replace(" ","_")}_gap_filler_value',
            'OldValue1' : 'NewValue1',
            'OldValue2' : 'NewValue2',
            'OldValue3' : 'NewValue3'}
    yam_dict[url] = column_specific_value_dict

    
    
with open('overrides_dict.yaml', 'w') as yam:
    dump(yam_dict, yam)


In [55]:
generic_from_yam

{'FILL_NA': 'Value_gap_filler_value',
 'OldValue1': 'NewValue1',
 'OldValue2': 'NewValue2',
 'OldValue3': 'NewValue3'}